In [1]:
import math
import time
import pandas as pd

# Define the false position function
def falsePosition(f, x0, x1, e=(10**(-10))):
    step = 1
    condition = True
    while condition:
        x2 = x0 - (x1-x0) * f(x0)/(f(x1) - f(x0))
        if f(x0) * f(x2) < 0:
            x1 = x2
        else:
            x0 = x2
        step += 1
        condition = abs(f(x2)) > e

    return x2, f(x2), step, x0, x1

# Read the file and store the equations and the intervals in two lists
functions = []
intervals = []
with open('functions.txt', 'r') as file:
    for line in file:
        equation, a, b = line.split(',')  # Split the line by commas
        # Convert the equation to a lambda function and append to the list
        functions.append(eval(f'lambda x: {equation}'))
        # Convert the bounds to floats and append to the list
        intervals.append([float(a), float(b)])

# Define the number of runs for each function
num_runs = 500

# Create a list of tuples containing the function, the interval, and the average time for the false position method
results = []
for i in range(10):
    f = functions[i]  # Get the function from the list
    # Get the interval from the list and use different variables
    a1, b1 = intervals[i]
    total_time = 0
    for j in range(num_runs):
        start_time = time.time()
        # Use the new variables as arguments
        result, fx, iter, x0, x1 = falsePosition(f, a1, b1)
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time

    average_time = total_time / num_runs
    # Use the new variables in the list
    results.append((f, [a1, b1], average_time))

# Create a pandas dataframe from the results list
df = pd.DataFrame(results, columns=['Function', 'Interval', 'Avg CPU Time'])

# Add columns for the root, the function value, the iterations, the lower bound, and the upper bound, using the falsePosition function
df['Root'] = df.apply(lambda row: falsePosition(row['Function'], row['Interval'][0], row['Interval'][1])[0], axis=1)
df['f(x)'] = df.apply(lambda row: falsePosition(row['Function'],row['Interval'][0], row['Interval'][1])[1], axis=1)
df['Iter'] = df.apply(lambda row: falsePosition(row['Function'], row['Interval'][0], row['Interval'][1])[2], axis=1)
# Add the lower bound column
df['a'] = df.apply(lambda row: falsePosition(row['Function'], row['Interval'][0], row['Interval'][1])[3], axis=1)
# Add the upper bound column
df['b'] = df.apply(lambda row: falsePosition(row['Function'], row['Interval'][0], row['Interval'][1])[4], axis=1)

# Drop the function and interval columns
df = df.drop(['Function', 'Interval'], axis=1)

# Add the problem column with labels
df['Problem'] = [f'$P{i+1}$' for i in range(10)]

# Reorder the columns
# Include the lower and upper bound columns
df = df[['Problem', 'Iter', 'Avg CPU Time', 'Root', 'f(x)', 'a', 'b']]

# Rename the columns
df = df.rename(columns={'Root': 'Approximate Root', 'f(x)': 'Function Value', 'a': 'Lower Bound', 'b': 'Upper Bound'})

# Use the scientific notation format for the Avg CPU Time column
pd.options.display.float_format = ' {:.12e}'.format

# Save the dataframe to an excel file with a different name, such as false-position-table.xlsx
df.to_excel('false-position-table.xlsx', index=False)

# Display the dataframe in Jupyter Notebook
df

,Problem,Iter,Avg CPU Time,Approximate Root,Function Value,Lower Bound,Upper Bound
0,$P1$,60,3.126893043518e-04,1.365230013410e+00,-7.482370278922e-11,1.365230013410e+00,4.000000000000e+00
1,$P2$,25,7.862138748169e-05,1.999999999986e+00,-5.665157232215e-11,1.999999999986e+00,4.000000000000e+00
2,$P3$,37,9.258985519409e-05,6.931471805263e-01,-6.726796897283e-11,6.931471805263e-01,2.000000000000e+00
3,$P4$,7,0.000000000000e+00,3.141592653590e+00,-1.299956738062e-13,3.141592013693e+00,3.141592653590e+00
4,$P5$,2,0.000000000000e+00,1.000000000000e+00,0.000000000000e+00,1.000000000000e+00,2.500000000000e+00
5,$P6$,22,1.158409118652e-04,-1.999999999904e+00,-9.559908420442e-11,-2.500000000000e+00,-1.999999999904e+00
6,$P7$,9,3.125143051147e-05,7.390851331711e-01,7.378908595257e-11,7.390851331711e-01,1.000000000000e+00
7,$P8$,23,9.375810623169e-05,2.999999999994e+00,-3.242472956799e-11,2.999999999994e+00,4.000000000000e+00
8,$P9$,2,0.000000000000e+00,0.000000000000e+00,0.000000000000e+00,0.000000000000e+00,1.000000000000e+00
9,$P10$,9,8.110284805298e-05,3.111748656309e+00,-1.332267629550e-12,3.051682692308e+00,3.111748656309e+00
